# Tool use (structured output) w/ Anthropic Claude 3.x versions on Amazon Bedrock

In this notebook, we show extracting structured output from the following use-cases:

1. **Text Summarization**
2. **Named Entity Recognition**
3. **Sentiment Analysis**
3. **Text Classification**

In [ ]:
%%bash
pip install uv && uv pip install -U sagemaker boto3 rich bs4 "anthropic[bedrock]"

In [1]:
!python --version

Python 3.10.14


In [ ]:
# restart kernel
from IPython.core.display import HTML

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [2]:
%load_ext rich

In [3]:
import json
import types
import requests
from datetime import date

import boto3
from anthropic import AI_PROMPT, HUMAN_PROMPT, AnthropicBedrock
from bs4 import BeautifulSoup
from requests.exceptions import JSONDecodeError, RequestException
from rich import print
from sagemaker import Session, get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## List Anthropic Model IDs in Bedrock

In [4]:
sess = Session()
region = sess.boto_region_name
boto_session = boto3.Session(region_name=region)
role = get_execution_role()

# Create bedrock runtime client
bedrock_runtime = boto3.client(service_name="bedrock-runtime", region_name=region)

# List Anthropic models in Bedrock
bedrock = boto3.client("bedrock", region_name=region)
models = bedrock.list_foundation_models(byProvider="Anthropic", byOutputModality="TEXT")[
    "modelSummaries"
]

# Print only Claude 3 models
model_ids = [
    model["modelId"]
    for model in models
    if "claude-3" in model["modelId"] and "v1:0:" not in model["modelId"]
]

print("Claude 3 ModelIDs")
print(model_ids)

print("===" * 10)
# change string in if loop for sonnet or opus
claude_haiku = [m for m in model_ids if "haiku" in m][0]
print(f"Haiku Model ID: [b red]{claude_haiku}")
print("===" * 10)

Claude 3 ModelIDs

[
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-opus-20240229-v1:0'
]

==============================

Haiku Model ID: anthropic.claude-3-haiku-20240307-v1:0

==============================

### Helper function to invoke LLM with bedrock runtime.

In [5]:
def chat_with_claude(prompt, tools, model_id=claude_haiku, bedrock_runtime=bedrock_runtime):
    """Function to invoke claude on Amazon Bedrock"""
    body = json.dumps(
        {
            "max_tokens": 4096,
            "messages": [{"role": "user", "content": prompt}],
            "tools": tools,
            "anthropic_version": "bedrock-2023-05-31",
        }
    )
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get("body").read())
    response_body_ns = types.SimpleNamespace(**response_body)
    return response_body_ns

## 1. Summarize Article

In this example, we'll use Claude to generate a JSON summary of an article, including fields for the author, topics, summary, coherence score, persuasion score, and a counterpoint.

In [6]:
tools = [
    {
        "name": "print_summary",
        "description": "Prints a summary of the article.",
        "input_schema": {
            "type": "object",
            "properties": {
                "author": {"type": "string", "description": "Name of the article author"},
                "topics": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": 'Array of topics, e.g. ["tech", "politics"]. Should be as specific as possible, and can overlap.',
                },
                "summary": {
                    "type": "string",
                    "description": "Summary of the article. One or two paragraphs max.",
                },
                "coherence": {
                    "type": "integer",
                    "description": "Coherence of the article's key points, 0-100 (inclusive)",
                },
                "persuasion": {
                    "type": "number",
                    "description": "Article's persuasion score, 0.0-1.0 (inclusive)",
                },
            },
            "required": ["author", "topics", "summary", "coherence", "persuasion", "counterpoint"],
        },
    }
]

# url = "https://www.anthropic.com/news/third-party-testing"
url = "https://aws.amazon.com/blogs/machine-learning/meta-llama-3-1-models-are-now-available-in-amazon-sagemaker-jumpstart/"
try:
    response = requests.get(url)
    response.raise_for_status()  # Raises an HTTPError for bad responses
except RequestException as e:
    print(f"Request error: {e}")

soup = BeautifulSoup(response.text, "html.parser")
article = " ".join([p.text for p in soup.find_all("p")])

summary_prompt = f"""
<article>
{article}
</article>

Use the `print_summary` tool.
"""

summary_response = chat_with_claude(summary_prompt, tools)

json_summary = None
for summary_content in summary_response.content:
    content = types.SimpleNamespace(**summary_content)
    if content.type == "tool_use" and content.name == "print_summary":
        json_summary = content.input
        break

if json_summary:
    print("JSON Summary")
    print(json.dumps(json_summary, indent=2))
else:
    print("No JSON Summary found in response")

JSON Summary

{
  "author": "Amazon Web Services",
  "topics": [
    "tech",
    "AI"
  ],
  "summary": "The article announces the availability of the Llama 3.1 collection of multilingual large language 
models (LLMs) through Amazon SageMaker JumpStart. Llama 3.1 includes pre-trained and instruction tuned generative 
AI models in 8B, 70B, and 405B sizes. It provides details on how to discover and deploy these Llama 3.1 models 
using SageMaker JumpStart, and highlights key features like long context length, support for grouped query 
attention, and optimization for multilingual dialogue use cases. The article also discusses the responsible use 
guide from Meta, and how SageMaker JumpStart provides secure deployment of these models within a virtual private 
cloud. Sample code is provided to demonstrate deploying and using the Llama 3.1 models.",
  "coherence": 5,
  "persuasion": 4,
  "counterpoint": "The article does not address potential limitations or drawbacks of the Llama 3.1 models, such as
potential biases or safety concerns that may arise from their use."
}

## 2. Named Entity Recognition

In this example, we'll use Claude to perform named entity recognition on a given text and return the entities in a structured JSON format.

In [ ]:
tools = [
    {
        "name": "print_entities",
        "description": "Prints extract named entities.",
        "input_schema": {
            "type": "object",
            "properties": {
                "entities": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "The extracted entity name."},
                            "type": {
                                "type": "string",
                                "description": "The entity type (e.g., PERSON, ORGANIZATION, LOCATION).",
                            },
                            "context": {
                                "type": "string",
                                "description": "The context in which the entity appears in the text.",
                            },
                        },
                        "required": ["name", "type", "context"],
                    },
                }
            },
            "required": ["entities"],
        },
    }
]

text = "John works at Google in New York. He met with Sarah, the CEO of Acme Inc., last week in San Francisco."

prompt = f"""
<document>
{text}
</document>

Use the print_entities tool.
"""

ner_response = chat_with_claude(prompt, tools, claude_haiku, bedrock_runtime)
json_entities = None
for content in ner_response.content:
    content = types.SimpleNamespace(**content)
    if content.type == "tool_use" and content.name == "print_entities":
        json_entities = content.input
        break

if json_entities:
    print("Extracted Entities (JSON):")
    print(json.dumps(json_entities, indent=4))
else:
    print("No entities found in the response.")

## 3. Sentiment Analysis

In this example, we'll use Claude to perform sentiment analysis on a given text and return the sentiment scores in a structured JSON format.

In [7]:
tools = [
    {
        "name": "print_sentiment_scores",
        "description": "Prints the sentiment scores of a given text.",
        "input_schema": {
            "type": "object",
            "properties": {
                "positive_score": {
                    "type": "number",
                    "description": "The positive sentiment score, ranging from 0.0 to 1.0.",
                },
                "negative_score": {
                    "type": "number",
                    "description": "The negative sentiment score, ranging from 0.0 to 1.0.",
                },
                "neutral_score": {
                    "type": "number",
                    "description": "The neutral sentiment score, ranging from 0.0 to 1.0.",
                },
            },
            "required": ["positive_score", "negative_score", "neutral_score"],
        },
    }
]

text = "The product was okay, but the customer service was terrible. I probably won't buy from them again."

prompt = f"""
<text>
{text}
</text>

Use the print_sentiment_scores tool.
"""

sent_response = chat_with_claude(prompt, tools)
json_sentiment = None
for sent_content in sent_response.content:
    # print(content)
    content = types.SimpleNamespace(**sent_content)
    if content.type == "tool_use" and content.name == "print_sentiment_scores":
        json_sentiment = content.input
        break

if json_sentiment:
    print("Sentiment Analysis (JSON):")
    print(json.dumps(json_sentiment, indent=2))
else:
    print("No sentiment analysis found in the response.")

Sentiment Analysis (JSON):

{
  "positive_score": 0.193,
  "negative_score": 0.487,
  "neutral_score": 0.32
}

## 4. Text Classification

In this example, we'll use Claude to classify a given text into predefined categories and return the classification results in a structured JSON format.

In [8]:
tools = [
    {
        "name": "print_classification",
        "description": "Prints the classification results.",
        "input_schema": {
            "type": "object",
            "properties": {
                "categories": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {"type": "string", "description": "The category name."},
                            "score": {
                                "type": "number",
                                "description": "The classification score for the category, ranging from 0.0 to 1.0.",
                            },
                        },
                        "required": ["name", "score"],
                    },
                }
            },
            "required": ["categories"],
        },
    }
]

text = "The new quantum computing breakthrough could revolutionize the tech industry."

classify_prompt = f"""
<document>
{text}
</document>

Use the print_classification tool. The categories can be Politics, Sports, Technology, Entertainment, Business.
"""

classify_response = chat_with_claude(classify_prompt, tools)
json_classification = None
for classify_content in classify_response.content:
    content = types.SimpleNamespace(**classify_content)
    if content.type == "tool_use" and content.name == "print_classification":
        json_classification = content.input
        break

if json_classification:
    print("Text Classification (JSON):")
    print(json.dumps(json_classification, indent=2))
else:
    print("No text classification found in the response.")

Text Classification (JSON):

{
  "categories": [
    {
      "name": "Politics",
      "score": 0.1
    },
    {
      "name": "Sports",
      "score": 0.1
    },
    {
      "name": "Technology",
      "score": 0.7
    },
    {
      "name": "Entertainment",
      "score": 0.1
    },
    {
      "name": "Business",
      "score": 0.3
    }
  ]
}